In [92]:
import torch
import torch.onnx
import torch.nn as nn
from torch import optim
from torchvision import models
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from sklearn.metrics import precision_score, recall_score, f1_score

In [93]:
hushem = './backend/datasets/HuSHem/'
hushem_data_folder = datasets.ImageFolder(hushem)

In [94]:
# Split the data into training and test sets
train_size = int(0.8 * len(hushem_data_folder))
test_size = len(hushem_data_folder) - train_size
train_data, test_data = random_split(hushem_data_folder, [train_size, test_size])

In [95]:
# Transformation des images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [96]:
# # Charger les images avec ImageFolder
# hushem_data = datasets.ImageFolder(root=hushem, transform=transform)

# # Créer le dataloader
# dataloader = DataLoader(hushem_data, batch_size=32, shuffle=True)

# # Vérifier si les données sont correctement chargées
# for inputs, labels in dataloader:
#     print(inputs.shape, labels.shape)
#     break

In [97]:
# Charger les images avec ImageFolder
hushem_data = datasets.ImageFolder(root=hushem, transform=transform)

# Séparer les données en ensembles d'entraînement et de test
train_size = int(0.8 * len(hushem_data))
test_size = len(hushem_data) - train_size
train_data, test_data = random_split(hushem_data, [train_size, test_size])

In [98]:
# Créer les dataloaders
train_dataloader = DataLoader(train_data, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=32, shuffle=False)


In [99]:
# Charger le modèle
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, len(hushem_data.classes))

c:\Users\teren\Documents\GitHub\Projet-AFH\venv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\teren\Documents\GitHub\Projet-AFH\venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [100]:
# Définir la fonction de perte
criterion = nn.CrossEntropyLoss()

In [101]:
# Choisir l'optimiseur
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [102]:
# Entraîner le modèle
num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

In [103]:
# Évaluer le modèle
model.eval()
with torch.no_grad():
    y_true = []
    y_pred = []
    for inputs, labels in test_dataloader:
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        y_true.extend(labels.tolist())
        y_pred.extend(preds.tolist())

In [104]:
# Calculer les métriques d'évaluation
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

In [109]:
# Afficher les résultats
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

Precision: 0.6506
Recall: 0.6319
F1 Score: 0.6183


___

Premier test avec entraînement sur 10 époques
- Precision: 0.5917
- Recall: 0.5824
- F1 Score: 0.5674

Deuxième test avec entraînement sur 30 époques
- Precision: 0.7217
- Recall: 0.7111
- F1 Score: 0.6839

___

Le modèle semble avoir obtenu de meilleures performances après avoir augmenté le nombre d'époques.Voici ce que signifient les métriques de précision, rappel et score F1 :

- **Précision (Precision) :** La précision mesure la proportion de vrais positifs parmi les prédictions positives totales. En d'autres termes, c'est la capacité du modèle à ne prédire comme positifs que les exemples qui le sont réellement.
    Dans notre cas, une **précision de 0.7217** signifie que près de **72.17 % des prédictions positives** faites par notre modèle sont effectivement correctes.

- **Rappel (Recall) :** Le rappel mesure la proportion de vrais positifs parmi les exemples réels positifs.
    C'est la capacité du modèle à identifier tous les exemples positifs.
    Avec un **rappel de 0.7111**, votre modèle a réussi à identifier environ **71.11 %** de tous les exemples positifs dans le jeu de données.

- **Score F1 :** Le score F1 est une mesure combinée de la précision et du rappel, calculée comme la moyenne harmonique entre ces deux métriques.
    Il est utile lorsque vous souhaitez trouver un équilibre entre la précision et le rappel.
    Un **score F1 de 0.6839** indique que votre modèle atteint un **bon équilibre** entre la précision et le rappel.

___

### Enregistrement du modèle

In [108]:
# Enregistrez le modèle
# Supposons que vous ayez un modèle appelé 'model' déjà entraîné et prêt à être exporté
dummy_input = torch.randn(32, 3, 224, 224)  # Remplacez ces valeurs par les dimensions de vos données
onnx_filename = "hushem_model.onnx"
# Exportez le modèle au format ONNX
torch.onnx.export(model, dummy_input, onnx_filename, verbose=True)

============== Diagnostic Run torch.onnx.export version 2.0.1+cpu ==============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [1]:
import onnxruntime as rt

In [2]:
sess = rt.InferenceSession('C:\\Users\\teren\\Documents\\GitHub\\Projet-AFH\\backend\\dashboard\\hushem_model.onnx')

In [3]:

# Obtenir des informations sur les entrées du modèle
inputs = sess.get_inputs()
for input in inputs:
    print(f"Nom de l'entrée: {input.name}")
    print(f"Type de l'entrée: {input.type}")
    print(f"Forme de l'entrée: {input.shape}")

Nom de l'entrée: input.1
Type de l'entrée: tensor(float)
Forme de l'entrée: [32, 3, 224, 224]


___